# Chapter 8 객체 참조, 가변성, 재활용

- 변수는 이름표임. 상자 자체가 아니다

## 변수는 상자가 아니다
 - 포스트잇으로 생각하는 것이 더욱 이해에 좋음
 - 변수가 객체에 할당된다 가 더 정확함 == 오른쪽 변이 먼저 생성됨
 - 변수는 레이블일 뿐이므로 객체에 여러 레이블을 붙일 수 있음 -> 별명이라고 함(nickname)

In [1]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))
        
x=Gizmo()

Gizmo id: 1953013207536


In [2]:
y = Gizmo() *10

Gizmo id: 1953013207344


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'get_ipython',
 'quit',
 'x']

## 정체성, 동질성, 별명


In [4]:
charles = {'name':'Charles L. Dodgson', 'born':1832}
lewis = charles
lewis is charles

True

In [5]:
id(charles), id(lewis) # 포스트잇 붙이듯이 둘이 같음

(1953013179904, 1953013179904)

In [6]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [7]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex ==charles

True

In [8]:
alex is charles # 동일한 값을 같지만 정체성은 다름

False

is 연산자를 사용할 때는 변수를 None 과 비교하는 것이 일반적임
singleton과 변수를 비교할 때는 is 를 써야함
is 는 오버로딩할 수 없으므로 파이썬이 특별 메서드를 호출할 필요가 없음
a==b 는 a.__eq__(b) 의 편리구문

### 튜플의 상대적 불변성
참조된 항목이 가변형이면 튜플 자체는 불변형이지만 참조된 항목은 변할 수 있다.
튜플 안에서 결코 변경되지 않는 것은 튜플이 담고있는 항목들의 정체성 뿐임

In [16]:
t1 = (1,2,[30,40]) # t1[-1] 은 가변형임
t2 = (1,2,[30,40])
t1 ==t2


True

In [17]:
t1 is t2

False

In [10]:
id(t1[-1])

1953012944064

In [12]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99, 99])

In [13]:
id(t1[-1]) # 가변항목이 바뀌었는데 정체성은 그대로임

1953012944064

In [14]:
t1==t2

False

## 기본 복사는 얕은 복사
- 리스트 및 가변형 시퀀스의 경우 사본을 생성함
그러나 생성자나 [:] 을 사용하면 얕은 사본을 생성함(shallow copy)
www.pythontutor.com 에서 예제 8-6 확인

## 깊은복사와 얕은 복사
 - 내포된 객체의 참조를 공유하지 않도록 깊게 복사할 필요가 있는 경우 copy 모듈의 deepcopy()
 - 얕은 복사는 copy()
 

In [27]:
class Bus:
    
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [28]:
import copy
bus1 = Bus(['A', 'B', 'C', 'D'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(1953014126864, 1953014190288, 1953014190192)

In [29]:
bus1.drop('B')
bus2.passengers

['A', 'C', 'D']

In [32]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(1953014151296, 1953014151296, 1953013180416)

In [31]:
bus3.passengers

['A', 'B', 'C', 'D']

In [33]:
a=[10,20]
b=[a,30]
a.append(b)
a

[10, 20, [[...], 30]]

In [34]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

## 참조로서의 함수 매개변수
- 파이썬은 공유로 호출(call by sharing) 하는 매개변수 전달 방식만 지원함
- 공유로 호출한다는 뜻:함수의 각 매개변수가 인수로 전달받은 각 참조의 사본을 받는다는 뜻
- 함수 안의 매개변수는 실제 인수의 별명 -> 가변형 객체 변경 가능

In [35]:
def f(a,b):
    a += b
    return a

In [36]:
x = 1
y = 2
f(x,y)

3

In [37]:
x,y

(1, 2)

In [38]:
a = [1, 2]
b = [3,4]
f(a,b)

[1, 2, 3, 4]

In [39]:
a,b # list a 는 변경됨

([1, 2, 3, 4], [3, 4])

In [40]:
t=(10,20)
u=(30,40)
f(t,u)

(10, 20, 30, 40)

In [41]:
t,u

((10, 20), (30, 40))

### 가변형을 매개변수 기본값으로 사용하기 - 좋지 않음

- 가변 값을 받는 매개변수의 기본값으로 None을 많이 사용함

In [42]:
class HauntedBus:
    
    def __init__(self, passengers = []): #passengers 인수를 전달하지 않는 경우 이 매개변수는 기본값인 빈 리스트에 바인딩
        self.passengers = passengers #passengers 인수를 전달하지 않는 경우 빈 리스트에 대한 별명으로 설정
            
    def pick(self, name): 
        self.passengers.append(name) #가변형 기본 리스트 변경하는 것임
    
    def drop(self, name):
        self.passengers.remove(name)

In [43]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [44]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [45]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [46]:
bus3 = HauntedBus()
bus3.passengers # 기본값이 비어있지 않다!

['Carrie']

In [47]:
bus3.pick('Dave')
bus2.passengers #bus 2도 같은 리스트를 참조 중

['Carrie', 'Dave']

In [48]:
bus2.passengers is bus3.passengers

True

In [49]:
bus1.passengers #bus1 은 별개의 리스트

['Bill', 'Charlie']

In [50]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [51]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [52]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### 가변 매개변수에 대한 방어적 프로그래밍
 - 중요한 것은 함수 구현자와 함수 호출자가 예상하는 것을 일치시키는 것임
 

In [53]:
class TwilightBus:
    
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers  #함수의 별명을 함수 객체에 같이 할당함 -> 원래 인수를 변경함
            #self.passengers = list(passengers) # passengers list의 사본을 만들거나, list 가 아니면 list 로 변환함
            #불명확한 경우 무조건 사본을 만들자!
            
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [54]:
basketball_team = ['a', 'b', 'c']
bus = TwilightBus(basketball_team)
bus.drop('a')
bus.drop('c')
basketball_team

['b']

## del 과 가비지 컬렉션
- del은 이름을 제거하는 것임. 객체를 제거하는 게 아님
- 가비지 컬렉션은 참조 카운트에 기반함(reference count) -> 객체는 개수를 세고 있음(refcount)
- refcount 가 0 이 되면 -> __del__() 메서드 호출 -> 메모리 해제

In [61]:
import weakref
s1 = {1,2,3}
s2 = s1
def bye():
    print ('Gone with the wind...')
type(s1)

set

In [62]:
ender = weakref.finalize(s1, bye) #약한 참조를 갖고 있어 코드가 작동함
ender.alive

True

In [63]:
del s1 #객체에 대한 참조를 제거함
ender.alive

True

In [64]:
s2='spam'

Gone with the wind...


In [65]:
ender.alive

False

## 약한 참조
- 객체를 유지시키지 않으면서 객체를 참조할 수 있으면 도움이 되는 경우 : 캐시
- 약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조함 -> 참조 대상 객체를 참조 대상(referent)

In [74]:
a_set = {0,1}
wref = weakref.ref(a_set)
wref

<weakref at 0x000001C6B8B98630; to 'set' at 0x000001C6B8AAA4A0>

In [75]:
wref()

{0, 1}

In [76]:
a_set = {2,3,4}
wref()

{0, 1}

In [77]:
wref() is None

False

In [78]:
wref() is None # 예제랑 다른데?

False

In [81]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
        
    def __repr__(self):
        return "Cheese(%r)" %self.kind

In [82]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese  # for loop 변수인 cheese 는 전역 변수이므로 명시적으로 제거하기 전에는 사라지지 않음

sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [83]:
del catalog
sorted(stock.keys())

['Parmesan']

In [84]:
del cheese
sorted(stock.keys())

[]

### 약한 참조의 한계
- 리스트와 dict 객체는 참조대상이 될 수 없음. 서브클래스를 만들면 참조 가능
- set은 참조대상이 될 수 있음
- int 및 tuple은 절대 대상이 될수 없음 -> 내부 최적화 문제

In [85]:
class MyList(list):
    '''약한 참조의 대상이 될 수 있는 list 서브클래스'''
    
a_list = MyList(range(10))

wref_to_a_list = weakref.ref(a_list)

- 단순 할당문은 사본을 생성하지 않음
- 복합 할당 연산자는 왼쪽 변수가 불변 객체에 바인딩되어 있을 때는 객체를 새로 생성하고 가변 객체에 바인딩되었을 때는 기존 객체를 변경
- 기존 변수에 새로운 값을 할당하면 기존에 바인딩 되어 있던 객체를 변경하는게 아니라 새로운 객체에 바인딩 된것
- 만약 그 변수가 기존 객체를 참조하는 마지막 참조였다면 객체는 가비지 컬렉트
- 함수 매개변수는 별명으로 전달되므로, 함수는 인수로 전달받은 가변 객체를 모두 변경할 수 있음. 이를 막으려면 함수 안에서 사본을 생성하거나, 튜플을 전달하는 등 불변 객체를 사용해야 함
- 함수 매개변수의 기본값으로 가변 객체를 사용하는 것은 위험하다. 매개변수를 변경하면 기본값이 변경되어 나중에 함수가 영향을 받음